In [1]:
import sys  
import warnings
sys.path.insert(1, 'utils')
warnings.filterwarnings("ignore")
from performance_confidence_metrics import *
from delong_metrics import *
from optimal_threshold import _threshold_finder, print_measures
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, confusion_matrix
import os
from tqdm import tqdm

In [2]:
internal_df = pd.read_csv('Dataset/Internal/Internal_final.csv')
erasmus_df = pd.read_csv('Dataset/External/Erasmus_Final_CSV.csv')
frac_atlas = pd.read_csv('Dataset/Open-source/frac_atlas_results.csv')
paediatric_dataset = pd.read_csv('Dataset/Open-source/paediatric_results.csv')

In [3]:
import pandas as pd
body_parts = [
    'Pelvis', 'Hip', 'Femur' , 'Fibula and Tibia', 'Wrist'
]

def calculate_metrics(df, body_part):
    part_df = df[df['Final BP'] == body_part]
    part_df.reset_index(inplace=True, drop=True)
    threshold = _threshold_finder(part_df['fracture_gt'], part_df['fracture_score'])
    
    TP, FP, TN, FN = perf_measure(part_df['fracture_gt'], pred_formu(part_df['fracture_score'], threshold))
    
    roc_auc = roc_auc_ci(part_df['fracture_gt'], part_df['fracture_score'])
    
    sensitivity_point_estimate, specificity_point_estimate, \
    sensitivity_confidence_interval, specificity_confidence_interval, \
    ppv_point_estimate, ppv_confidence_interval, \
    npv_point_estimate, npv_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95)
    
    results_df = pd.DataFrame({
        'Body Part': [body_part.capitalize()],
        'Threshold': [threshold],
        'ROC AUC': [roc_auc],
        'Sensitivity': [sensitivity_point_estimate],
        'Specificity': [specificity_point_estimate],
        'Sensitivity CI': [sensitivity_confidence_interval],
        'Specificity CI': [specificity_confidence_interval]
    })
    
    return results_df

all_results_df = pd.concat([calculate_metrics(erasmus_df, part) for part in tqdm(body_parts)], ignore_index=True)
all_results_df.to_csv('Results_csv/Erasmus_performance.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 11.58it/s]


In [4]:
body_part_targets = {
    'forearm_intern': 'forearm',
    'foot_intern': 'foot',
    'humerus_intern': 'humerus',
    'pelvis_intern': 'pelvis',
    'leg_intern': 'leg',
    'ankle_intern': 'ankle',
    'clavicle_intern': 'clavicle',
    'knee_intern': 'knee',
    'fibula_intern': 'fibula tibia',
    'femur_intern': 'femur',
    'hip_intern': 'hip',
    'wrist_intern': 'wrist',
    'elbow_intern': 'elbow',
    'hand_intern': 'hand',
    'finger_intern': 'finger',
    'shoulder_intern': 'shoulder',
    'toe_intern': 'toe'
}

def calculate_metrics(df, target_col):
    threshold = _threshold_finder(df['fracture_target'], df['fracture'])
    TP, FP, TN, FN = perf_measure(df['fracture_target'], pred_formu(df['fracture'], threshold))
    roc_auc = roc_auc_ci(df['fracture_target'], df['fracture'])
    sensitivity_point_estimate, specificity_point_estimate, \
    sensitivity_confidence_interval, specificity_confidence_interval, \
    ppv_point_estimate, ppv_confidence_interval, \
    npv_point_estimate, npv_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95)
    

    results_df = pd.DataFrame({
        'Body Part': [target_col.capitalize()],
        'Threshold': [threshold],
        'ROC AUC': [roc_auc],
        'Sensitivity': [sensitivity_point_estimate],
        'Specificity': [specificity_point_estimate],
        'Sensitivity CI': [sensitivity_confidence_interval],
        'Specificity CI': [specificity_confidence_interval]
    })
    
    return results_df

all_results_df = pd.DataFrame()
for df_name, target_col in tqdm(body_part_targets.items()):
    part_df = internal_df[internal_df[target_col + '_target'] == 1].reset_index(drop=True)
    result_df = calculate_metrics(part_df, target_col)
    
    all_results_df = pd.concat([all_results_df, result_df], ignore_index=True)

all_results_df.to_csv('Results_csv/internal_bodypart_fracture_performance.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:12<00:00,  1.41it/s]


In [5]:
target_columns = [
    'femur', 'shoulder', 'humerus', 'elbow', 'knee', 'forearm',
    'fibula tibia', 'clavicle', 'toe', 'foot', 'wrist', 'hand',
    'leg', 'ankle', 'hip', 'finger', 'pelvis'
]


def calculate_metrics(df, target_col):
    threshold = _threshold_finder(df[target_col + '_target'], df[target_col])
    TP, FP, TN, FN = perf_measure(df[target_col + '_target'], pred_formu(df[target_col], threshold))
    roc_auc = roc_auc_ci(df[target_col + '_target'], df[target_col])
    sensitivity_point_estimate, specificity_point_estimate, \
    sensitivity_confidence_interval, specificity_confidence_interval, \
    ppv_point_estimate, ppv_confidence_interval, \
    npv_point_estimate, npv_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95)
    results_df = pd.DataFrame({
        'Body Part': [target_col.capitalize()],
        'Threshold': [threshold],
        'ROC AUC': [roc_auc],
        'Sensitivity': [sensitivity_point_estimate],
        'Specificity': [specificity_point_estimate],
        'Sensitivity CI': [sensitivity_confidence_interval],
        'Specificity CI': [specificity_confidence_interval]
    })
    
    return results_df

all_results_df = pd.DataFrame()

for target_col in tqdm(target_columns):
    result_df = calculate_metrics(internal_df, target_col)
    all_results_df = pd.concat([all_results_df, result_df], ignore_index=True)

all_results_df.to_csv('Results_csv/internal_body_parts_classification_performance.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [05:01<00:00, 17.76s/it]


In [6]:
body_part_targets = {
    'forearm_intern': 'forearm',
    'foot_intern': 'foot',
    'humerus_intern': 'humerus',
    'pelvis_intern': 'pelvis',
    'leg_intern': 'leg',
    'ankle_intern': 'ankle',
    'clavicle_intern': 'clavicle',
    'knee_intern': 'knee',
    'fibula_intern': 'fibula tibia',
    'femur_intern': 'femur',
    'hip_intern': 'hip',
    'wrist_intern': 'wrist',
    'elbow_intern': 'elbow',
    'hand_intern': 'hand',
    'finger_intern': 'finger',
    'shoulder_intern': 'shoulder',
    'toe_intern': 'toe'
}

def calculate_metrics(df, target_col):
    threshold = _threshold_finder(df['treated_fracture_target'], df['treated_fracture'])
    TP, FP, TN, FN = perf_measure(df['treated_fracture_target'], pred_formu(df['treated_fracture'], threshold))
    roc_auc = roc_auc_ci(df['treated_fracture_target'], df['treated_fracture'])
    sensitivity_point_estimate, specificity_point_estimate, \
    sensitivity_confidence_interval, specificity_confidence_interval, \
    ppv_point_estimate, ppv_confidence_interval, \
    npv_point_estimate, npv_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95)
    

    results_df = pd.DataFrame({
        'Body Part': [target_col.capitalize()],
        'Threshold': [threshold],
        'ROC AUC': [roc_auc],
        'Sensitivity': [sensitivity_point_estimate],
        'Specificity': [specificity_point_estimate],
        'Sensitivity CI': [sensitivity_confidence_interval],
        'Specificity CI': [specificity_confidence_interval]
    })
    
    return results_df

all_results_df = pd.DataFrame()
for df_name, target_col in tqdm(body_part_targets.items()):
    part_df = internal_df[internal_df[target_col + '_target'] == 1].reset_index(drop=True)
    result_df = calculate_metrics(part_df, target_col)
    
    all_results_df = pd.concat([all_results_df, result_df], ignore_index=True)
all_results_df.to_csv('Results_csv/internal_treated_fracture_performance.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:12<00:00,  1.33it/s]


In [7]:
target_columns = ['leg', 'hand', 'hip', 'shoulder']
def calculate_metrics(df, target_col):
    part_df = df[df[target_col] == 1]
    part_df.reset_index(inplace=True, drop=True)
    threshold = print_measures(part_df['fractured'], part_df['fracture_average'])
    TP, FP, TN, FN = perf_measure(part_df['fractured'], pred_formu(part_df['fracture_average'], threshold))
    roc_auc = roc_auc_ci(part_df['fractured'], part_df['fracture_average'])
    sensitivity_point_estimate, specificity_point_estimate, \
    sensitivity_confidence_interval, specificity_confidence_interval, \
    ppv_point_estimate, ppv_confidence_interval, \
    npv_point_estimate, npv_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95)
    
    results_df = pd.DataFrame({
        'Body Part': [target_col.capitalize()],
        'Threshold': [threshold],
        'ROC AUC': [roc_auc],
        'Sensitivity': [sensitivity_point_estimate],
        'Specificity': [specificity_point_estimate],
        'Sensitivity CI': [sensitivity_confidence_interval],
        'Specificity CI': [specificity_confidence_interval]
    })
    
    return results_df

all_results_df = pd.DataFrame()

for target_col in tqdm(target_columns):
    result_df = calculate_metrics(frac_atlas, target_col)
    all_results_df = pd.concat([all_results_df, result_df], ignore_index=True)

all_results_df.to_csv('Results_csv/FracAtlas_classification_performance.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 39.20it/s]


In [8]:
def calculate_pediatric_metrics(df, subgroup_name):
    threshold = print_measures(df['fracture_visible'], df['fracture_average'])
    TP, FP, TN, FN = perf_measure(df['fracture_visible'], pred_formu(df['fracture_average'], threshold))
    roc_auc = roc_auc_ci(df['fracture_visible'], df['fracture_average'])
    
    sensitivity_point_estimate, specificity_point_estimate, \
    sensitivity_confidence_interval, specificity_confidence_interval, \
    ppv_point_estimate, ppv_confidence_interval, \
    npv_point_estimate, npv_confidence_interval \
        = sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95)
    
    results_df = pd.DataFrame({
        'Subgroup': [subgroup_name],
        'Threshold': [threshold],
        'ROC AUC': [roc_auc],
        'Sensitivity': [sensitivity_point_estimate],
        'Specificity': [specificity_point_estimate],
        'Sensitivity CI': [sensitivity_confidence_interval],
        'Specificity CI': [specificity_confidence_interval]
    })
    
    return results_df

subgroups = [
    (paediatric_dataset[paediatric_dataset['age'] >= 10], 'More than 10 years old'),
    (paediatric_dataset[paediatric_dataset['age'] <= 9], 'Less than 9 years old'),
    (paediatric_dataset[paediatric_dataset['gender'] == 'M'], 'Male'),
    (paediatric_dataset[paediatric_dataset['gender'] == 'F'], 'Female'),
    (paediatric_dataset[paediatric_dataset['device_manufacturer'] == 'Siemens'], 'Siemens'),
    (paediatric_dataset[paediatric_dataset['device_manufacturer'] == 'Agfa'], 'Agfa'),
    (paediatric_dataset[paediatric_dataset['projection'] == 2], 'Lateral Projection'),
    (paediatric_dataset[paediatric_dataset['projection'] == 1], 'PA Projection'),
    (paediatric_dataset[paediatric_dataset['projection'] == 3], 'Oblique Projection')
]

all_pediatric_results_df = pd.DataFrame()

for subgroup, name in tqdm(subgroups):
    result_df = calculate_pediatric_metrics(subgroup.reset_index(drop=True), name)
    all_pediatric_results_df = pd.concat([all_pediatric_results_df, result_df], ignore_index=True)

all_pediatric_results_df.to_csv('Results_csv/Pediatric_Subgroups_Performance.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 13.64it/s]
